In [51]:
import cv2
import numpy as np
from skimage import filters
import sys


# Load the image
image_path = r'C:\images\img7.jpg'

original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# blurred_image = cv2.GaussianBlur(original_image, (5,5), 0)

_, binary_image = cv2.threshold(original_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# binary_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 20)

# kernel = np.ones((3, 3), np.uint8)
# binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=1)


# Save the preprocessed image
cv2.imwrite('preprocessed_image.jpg', binary_image)

cv2.imshow('Original Image', original_image)
cv2.imshow('Binarized Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:

# Load the pre-trained EAST text detector model and configure the parameters
east_model_path = "frozen_east_text_detection.pb"  # Path to the EAST model file
min_confidence = 0.5  # Minimum probability confidence threshold to consider a region as text
width, height = 320, 320  # Image dimensions (should be a multiple of 32)

# Load the input image
image_path = "preprocessed_image.jpg"
image = cv2.imread(image_path)
orig = image.copy()

# Prepare the image for text detection (resizing and normalization)
blob = cv2.dnn.blobFromImage(image, 1.0, (width, height), (123.68, 116.78, 103.94), swapRB=True, crop=False)

# Load the pre-trained EAST text detector model
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

# Set the prepared blob as the input to the network and perform a forward pass
net.setInput(blob)
(scores, geometry) = net.forward(["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"])

# Decode the predictions and obtain the bounding box coordinates
rectangles, confidences = cv2.dnn.NMSBoxesRotated(
    boxes_list=geometry,
    confidences=scores,
    score_threshold=min_confidence,
    nms_threshold=0.4
)

# Loop over the bounding boxes and draw them on the image
for i in range(len(rectangles)):
    # Get the coordinates of the bounding box for the text region
    (rW, rH) = (geometry[i][2], geometry[i][3])
    (dX, dY) = (geometry[i][0], geometry[i][1])
    angle = geometry[i][4]

    # Calculate the box dimensions and coordinates
    (startX, startY) = (int(dX * width), int(dY * height))
    (endX, endY) = (int(rW * width), int(rH * height))
    box = ((startX, startY), (endX, startY), (endX, endY), (startX, endY), angle)

    # Draw the bounding box on the image
    cv2.polylines(orig, [box], isClosed=True, color=(0, 255, 0), thickness=2)

# Display the output image with bounding boxes around text regions
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\tensorflow\tf_importer.cpp:2999: error: (-215:Assertion failed) netBinSize || netTxtSize in function 'cv::dnn::dnn4_v20230620::`anonymous-namespace'::TFImporter::populateNet'


In [53]:
import cv2
import numpy as np

# Load the preprocessed image
image_path = 'preprocessed_image.jpg'  # Replace with your preprocessed image path
preprocessed_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Perform edge detection
edges = cv2.Canny(preprocessed_image, threshold1=30, threshold2=100)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the original image
image_with_contours = cv2.imread(image_path)

# Loop through the detected contours
for contour in contours:
    # Get the coordinates and dimensions of the bounding rectangle around the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Draw a green rectangle around the detected text region
    cv2.rectangle(image_with_contours, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with detected text regions
cv2.imshow("Text Detection", image_with_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [54]:
import cv2

# Load the preprocessed image (preprocessed_image.jpg is the output from the preprocessing step)
preprocessed_image_path = 'preprocessed_image.jpg'
preprocessed_image = cv2.imread(preprocessed_image_path, cv2.IMREAD_GRAYSCALE)

# Perform edge detection using Canny
edges = cv2.Canny(preprocessed_image, threshold1=30, threshold2=100)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area to find potential text regions
min_contour_area = 100  # Minimum contour area to be considered as text region
text_regions = []
for contour in contours:
    if cv2.contourArea(contour) > min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)
        text_regions.append((x, y, w, h))

# Draw rectangles around the detected text regions on the original image
original_image = cv2.imread('preprocessed_image.jpg')  # Replace with the path to your original image
for (x, y, w, h) in text_regions:
    cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with detected text regions
cv2.imshow('Text Detection', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
